In [2]:
import sys
import os
from os import path
# 部署的路径
sys.path.append('/root/bdrisk/risk_project')
# 堡垒机的路径
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
# 这是log文件的存放路径
root_path = path.dirname(path.dirname(path.dirname(os.getcwd())))
from risk_models import *
# from risk_models import 
import datetime
import numpy as np

# 先录入70家企业除上飞

In [12]:
sql=f'''select UNI_SC_ID,LONGITUDE,LATITUDE from dim.DIM_FTZ_CORP
WHERE UNI_SC_ID = '91310000132612172J' '''
df_ftz = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-06-24 15:19:32.698 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.072s


In [13]:
ftz_list = df_ftz.to_dict('records')

In [14]:
def get_tag(tagid):
#     df_tag
    try:
        tag = df_tag.loc[df_tag['GP_FTTAGCONFIG_FK']==tagid,'TAG_VALUE'].iloc[0]
    except:
        tag = ''
#     print(tag)
    return tag
def dw_corp_info():
    """
    Des
        DW层的CORP_INFO数据表
    """
    sql = f"""
    select CORP_NAME, CORP_TYPE,PERSON_NAME,ADDRESS,AREA_CODE,ZIP,TELEPHONE,ESTABLISH_DATE,
    CURRENCY,to_char(BUSINESS_SCOPE),PERSON_CERT_TYPE,PERSON_CERT_CODE,INDUSTRY_CODE,REG_NO,
    RECEIVING_ORGAN,CHANGE_DATE,CHANGE_ITEM,BUSINESS_ADDRESS,UNI_SC_ID,CORP_STATUS,REG_CAPITAL,
    PERSON_EMAIL,PERSON_LANDLINE_TEL,SUB_OBJ,CONTACT_MOBILE,CORP_COUNTRY,CONTACT_EMAIL,
    CONTACT_CER_TYPE,CONTACT_CER_NO
    from dw_lgxc_basic.ft_gov_dtl_corp_info
    where UNI_SC_ID = '{org_code}'
    """
    df = Read_Oracle().read_oracle(sql=sql, database='dbdw')
    return df
def ods_CBRS():
    """
    Des
        ODS层的参保人数数据
        # DWCBRS 参保人数 
        # DWMC 单位名称
        # JNJE 缴纳金额
        # SFQJ 是否欠缴
        # CBNY 年月
        # ZZZGPJRS 在职职工年平均人数
    """
    sql = """
    select DWCBRS, DWMC, JNJE, SFQJ, CBNY, ZZZGPJRS
    from DW_LGXC_BASIC.FT_GOV_DTL_ZWY_DWCBQK_XXB
    """
    df = Read_Oracle().read_oracle(sql=sql, database='dbdw')
    return df
def dw_dim_industry_code():
    """
    Des
        获取行业代码
        # HY_DM 行业代码
        # HYMC 行业类型
    """
    sql = """
    select *
    from dim.dim_dm_gy_hy
    """
    return Read_Oracle().read_oracle(sql=sql, database='dbdw') 
def ods_corp_info_ext():
    """
    Des
        获取企业信息
    """
    
    sql="""
    select *
    from DW_LGXC_BASIC.FT_GOV_DTL_CORP_EXT
    """
    return Read_Oracle().read_oracle(sql=sql, database='dbdw')
def ods_tax_level():
    """
    Des
        获取企业的纳税等级
    """
    
    sql='''
    SELECT
        t2.NSRMC,
        t2.SHXYDM,
        t1.PDDJ 
        FROM DW_LGXC_BASIC.FT_GOV_DTL_SW_XYDJ t1
        LEFT JOIN DW_LGXC_BASIC.FT_GOV_DTL_SW_DJ_NSRXX t2
        ON t1.DJXH = t2.DJXH
        WHERE PDND=2018
    '''
    return Read_Oracle().read_oracle(sql=sql, database='dbdw')
def ods_cos_level():
    sql='''
    SELECT
      v.CORP_NAME,
      T.UNI_SC_ID,
      MTYPE FROM ODS_ZMXPQ.VIEW_CUST_MTYPE v
    LEFT JOIN DIM.map_ftz_corp_id t
      ON v.CORP_UID = t.CORP_UID
    WHERE v.REPEAL_DATE IS NULL'''
    return Read_Oracle().read_oracle(sql=sql, database='dbods')

In [15]:
def model_dp4_corp_info(org_code):
    global df_tag
    sql=f'''select * from DW_CORP_BASIC.ft_tag_corp  A LEFT JOIN DW_CORP_BASIC.DW_CORP_UID B
    ON A.ENT_BDC_UNI_ID = B.ENT_BDC_UNI_ID
    where b.UNF_SC_ID ='{org_code}'
    and b.isdeleted = 0 '''
    df_tag = Read_Oracle().read_oracle(sql=sql, database='dbdw')
    # I_E_METHOD 进出口方式
    tag1 = get_tag(34)
    # CUS_QUA 报关资格
    tag2 = get_tag(43)
    # DANGER_GOODS 是否经营危险品
    tag3 = get_tag(45)
    # COLD_FOOD 是否从事冷链食品
    tag4 = get_tag(29)
    # EST_YEARS 成立年限
    tag5 = get_tag(39)
    # REG_CAPITAL 注册资本
    tag6 = get_tag(9)
    # I_E_TYPE 进出口方式
    tag7 = get_tag(28)
    # CORP_TYPE 企业类型
    tag8 = get_tag(10)
    # INDUSTRY 所属行业
    tag9 = get_tag(12)
    # TAX_RATE 纳税信用等级
    tag10 = get_tag(21)
    # CUS_RATE 海关信用等级
    tag11 = get_tag(22)
    # EXC_RATE 外汇管理信用等级
    tag12 = get_tag(20)
    def corp_info_main():
        # dw_crop_info_df
        dw_crop_info_df = dw_corp_info()
        # cbrs_df
        cbrs_df = ods_CBRS()
        # 关联DW与参保人数关联到企业名称
        cbrs_df.drop_duplicates(['DWMC'], keep='last', inplace=True)
        dw_crop_info_df_cb = pd.merge(dw_crop_info_df, cbrs_df, left_on='CORP_NAME', right_on='DWMC', how='left')
        dw_crop_info_df_cb.drop(['DWMC'], axis=1, inplace=True)

        # 获取企业的行业代码
        Industry_code_df = dw_dim_industry_code()
        Industry_code_df = Industry_code_df[['HY_DM', 'HYMC']].drop_duplicates()
        dw_crop_info_df_cb['INDUSTRY_CODE'] = dw_crop_info_df_cb['INDUSTRY_CODE'].map(lambda x: str(x)[1:])
        dw_crop_info_df_cb_indco = pd.merge(dw_crop_info_df_cb, Industry_code_df, left_on='INDUSTRY_CODE', right_on='HY_DM', how='left')
        dw_crop_info_df_cb_indco.drop(['HY_DM', 'INDUSTRY_CODE'], axis=1, inplace=True)

        dw_crop_info_df_cb_indco = dw_crop_info_df_cb_indco.rename(columns={'TO_CHAR(BUSINESS_SCOPE)': 'BUSINESS_SCOPE'})
        # 00010000 企业   00010100 公司     00010200 分公司
        dw_crop_info_df_cb_indco['CORP_TYPE'] = dw_crop_info_df_cb_indco['CORP_TYPE'].map({'00010100':'公司', '00010200':'分公司'})
        # RECEIVING_ORGAN 受理机关
        del dw_crop_info_df_cb_indco['RECEIVING_ORGAN']

        # 纳税等级
        tax_level = ods_tax_level()
        dw_crop_info_df_cb_indco_tax = pd.merge(dw_crop_info_df_cb_indco, tax_level, left_on='UNI_SC_ID', right_on='SHXYDM', how='left')
        dw_crop_info_df_cb_indco_tax.drop(['NSRMC', 'SHXYDM'], axis=1, inplace=True)
        # 海关信用等级
        cus_level = ods_cos_level()
        res = pd.merge(dw_crop_info_df_cb_indco_tax, cus_level[['UNI_SC_ID', 'MTYPE']], on='UNI_SC_ID', how='left')
        return res
    corp_info = corp_info_main()
    corp_info['I_E_METHOD'] = tag1
    corp_info['CUS_QUA'] = tag2
    corp_info['DANGER_GOODS'] = tag3
    corp_info['COLD_FOOD'] = tag4
    corp_info['EST_YEARS'] = tag5
    corp_info['REG_CAPITAL'] = tag6
    corp_info['I_E_TYPE'] = tag7
    corp_info['CORP_TYPE'] = tag8
    corp_info['INDUSTRY'] = tag9
    corp_info['TAX_RATE'] = tag10
    corp_info['CUS_RATE'] = tag11
    corp_info['EXC_RATE'] = tag12
    corp_info['FIRSTINSERTDT'] = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
    corp_info['FIRSTINSERTDT'] = pd.to_datetime(corp_info['FIRSTINSERTDT'], format='%Y-%m-%d %H:%M:%S')
    final = corp_info.reset_index().rename(columns={'index': 'ID'})
    final.rename(columns={'CORP_NAME':'ORG_NAME','UNI_SC_ID':'ORG_CODE'},inplace=True)
#     final['LON'] = i['LONGITUDE']
#     final['LAT'] = i['LATITUDE']
    final['LON']='121.859346'
    final['LAT']='31.087247'
    final = final[['ID','ORG_CODE','ORG_NAME','I_E_METHOD','CUS_QUA','DANGER_GOODS','COLD_FOOD','EST_YEARS','REG_CAPITAL','I_E_TYPE','CORP_TYPE','INDUSTRY','TAX_RATE','CUS_RATE','EXC_RATE','ADDRESS','LON','LAT','REG_NO','CORP_COUNTRY','SUB_OBJ','HYMC','BUSINESS_SCOPE','BUSINESS_ADDRESS','AREA_CODE','ZIP', 'TELEPHONE', 'ESTABLISH_DATE', 'CURRENCY','PERSON_NAME','PERSON_CERT_TYPE', 'PERSON_CERT_CODE','CORP_STATUS','PERSON_EMAIL', 'PERSON_LANDLINE_TEL','CHANGE_DATE','CHANGE_ITEM','CONTACT_MOBILE', 'CONTACT_EMAIL', 'CONTACT_CER_TYPE','CONTACT_CER_NO','DWCBRS','JNJE', 'SFQJ', 'CBNY', 'ZZZGPJRS','PDDJ', 'MTYPE']]
    final['CHECK_TIME'] = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
    final['CHECK_TIME'] = pd.to_datetime(final['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
    final.fillna('',inplace=True)
    Write_Oracle().write_oracle('BD_RISK_CORP_INFO_BASIC', final, org_code=org_code,alarm=None)
#     print(final.iloc[0])
#     return final

In [16]:
for i in ftz_list:
    org_code = i['UNI_SC_ID']
    model_dp4_corp_info(org_code=org_code)

2021-06-24 15:19:46.282 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.575s
2021-06-24 15:19:46.452 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.131s
2021-06-24 15:19:46.580 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.126s
2021-06-24 15:19:46.776 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.179s
2021-06-24 15:19:46.908 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.103s
2021-06-24 15:19:47.050 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.127s
2021-06-24 15:19:47.177 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 -

In [7]:
org_code='91310115688774070B'
model_dp4_corp_info(org_code=org_code)

2021-06-24 13:25:01.938 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.193s
2021-06-24 13:25:02.054 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.107s
2021-06-24 13:25:02.170 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.113s
2021-06-24 13:25:02.375 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.139s
2021-06-24 13:25:02.514 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.094s
2021-06-24 13:25:02.944 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.415s


,ID,ORG_CODE,ORG_NAME,I_E_METHOD,CUS_QUA,DANGER_GOODS,COLD_FOOD,EST_YEARS,REG_CAPITAL,I_E_TYPE,...,CONTACT_CER_TYPE,CONTACT_CER_NO,DWCBRS,JNJE,SFQJ,CBNY,ZZZGPJRS,PDDJ,MTYPE,CHECK_TIME
0,0,91310115688774070B,上海综合保税区国际物流有限公司,,,,,,,,...,身份证,31022519880902462X,65,220868.9,否,202104,0,B,高级认证企业,2021-06-24 13:25:02


In [9]:
sql='''select * from BD_RISK_CORP_INFO_BASIC where iscurrent = 1'''
corp_info = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-06-23 10:27:00.322 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.141s


In [11]:
corp_info.to_excel('corp_info.xlsx')